In [1]:
import pandas as pd

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim import corpora

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [39]:
from collections import Counter

In [66]:
add_stop = ['local', 'law', 'chair', 'land', 'comittee','application','oversight','committee','application','number','land use',
            'resolution','communication','new','york','city','council','councilmember','district','amend','code','administrative',
            'relation', 'calling', 'state', 'requiring', 'department', 'pass', 'sign', 'block', 'legislature', 'charter', 'governor',
             'section', 'property', 'lot', 'pursuant', 'approving', 'upon', 'establishing', 'borough', 'certain', 'commission', 'amendment', 
             'public', 'act', 'located', 'program', 'real', 'c', 'n','legislation', 'exemption', 'report', 'submitted','indtroduce']

In [55]:
def preprocess_data(documents):
    stop_words = set(stopwords.words('english'))
    stop_words.update(add_stop)

    tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]
    processed_docs = [[word for word in doc if word not in stop_words and word.isalpha()] for doc in tokenized_docs]
    return processed_docs

In [26]:
def top_terms_for_member(member, n=5):
    if member in tfidf_df.index:
        print(f"Top {n} terms for {member}:")
        member_tfidf = tfidf_df.loc[member]
        top_terms = member_tfidf.nlargest(n)
        for term in top_terms.index:
            print(term)
    else:
        print(f"No data for individual: {member}")

In [17]:
def join_tokens(token_list):
    return ' '.join(token_list)

#### Impot and clean legislation tables

In [12]:
try:
    leg_2024 = pd.read_csv("../data/cy2023csv.csv", encoding='ISO-8859-1')
except UnicodeDecodeError:
    try:
        leg_2024 = pd.read_csv("../data/cy2023csv.csv", encoding='cp1252')
    except UnicodeDecodeError:
        pass

In [13]:
try:
    leg_2023 = pd.read_csv('../data/cy2023csv.csv', encoding='ISO-8859-1')
except UnicodeDecodeError:
    try:
        leg_2023 = pd.read_csv('../data/cy2023csv.csv', encoding='cp1252')
    except UnicodeDecodeError:
        pass

In [56]:
all_leg = pd.concat([leg_2023, leg_2024])

In [57]:
all_leg['processed_text'] = all_leg['Title'].apply(lambda x: preprocess_data([x])[0])

In [58]:
all_leg = all_leg[['Type','Status','Committee','Prime Sponsor','processed_text']]
all_leg = all_leg.rename(columns={'Prime Sponsor':'prime_sponsor'})
all_leg = all_leg.dropna()

In [59]:
all_tokens = [token for sublist in all_leg['processed_text'] for token in sublist]
word_freq = Counter(all_tokens)

most_common_words = word_freq.most_common(40)  

In [60]:
common_words_list = [word for word, freq in most_common_words]
common_words_string = ', '.join(f"'{word}'" for word in common_words_list)
print(common_words_string)

'community', 'housing', 'services', 'zoning', 'street', 'school', 'education', 'health', 'private', 'map', 'tax', 'would', 'n', 'planning', 'bronx', 'provide', 'avenue', 'manhattan', 'use', 'designation', 'finance', 'lots', 'preconsidered', 'development', 'service', 'brooklyn', 'taxes', 'information', 'decision', 'reporting', 'within', 'building', 'prohibiting', 'repeal', 'schools', 'emergency', 'queens', 'office', 'mental', 'states'


#### tf-idf

In [61]:
all_leg['joined_text'] = all_leg['processed_text'].apply(join_tokens)

In [62]:
grouped_df = all_leg.groupby('prime_sponsor')['joined_text'].apply(' '.join).reset_index()

In [20]:
tfidf = TfidfVectorizer()

In [63]:
tfidf_matrix = tfidf.fit_transform(grouped_df['joined_text'])

In [64]:
feature_names = tfidf.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names, index=grouped_df['prime_sponsor'])

In [65]:
top_terms_for_member('Rita C. Joseph',5)

Top 5 terms for Rita C. Joseph:
leadership
teams
student
students
introduce


##### LDA